# Heavy Truck Model
## FAF Summarization

Author: Maddie Hasani, Fehr & Peers <br/>
Reviewer: Fatemeh Ranaiefar, Fehr & Peers<br/>
Last update: 11/17/2023


## REQUIRED LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter

In [2]:
# pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

## LOAD IN & CLEAN DATA

In [3]:
# 1. Load in some inputs
inputs_sandag_HTM = pd.ExcelFile('inputs_sandag_HTM_11.17.xlsx')
sheet_names = [sheet_name for sheet_name in inputs_sandag_HTM.sheet_names if sheet_name.lower() not in ['userguide', 'reference']]

# 1.1 Load all sheets into separate DataFrames with lowercase names
for sheet_name in sheet_names:
    df_name = sheet_name.lower()  # Convert sheet name to lowercase
    globals()[df_name] = inputs_sandag_HTM.parse(sheet_name)  # Save DataFrame to a variable with the lowercase name

# 2. Load in FAF data
faf_name = faf.loc[0, 'Name']
faf_path = faf.loc[0, 'Path']
full_faf_path = faf_path + '\\' + faf_name + ".csv"

faf = pd.read_csv(full_faf_path)
faf.head()

,dms_orig,dms_dest,Mode,Commodity,Direction,Trade,fr_orig,fr_dest,fr_inmode,fr_outmode,distons_2017,disvalue_2017,distons_2025,disvalue_2025,distons_2030,disvalue_2030,distons_2035,disvalue_2035,distons_2040,disvalue_2040,distons_2045,disvalue_2045,distons_2050,disvalue_2050
0,605901,605900,5,32,II,Domestic,NaN,NaN,NaN,NaN,0.010000,0.16,0.01000,0.18,0.010000,0.18,0.01,0.19,0.01,0.19,0.01,0.20,0.01,0.20
1,605900,605900,5,32,II,Domestic,NaN,NaN,NaN,NaN,0.110000,1.77,0.13000,1.97,0.130000,2.01,0.13,2.07,0.14,2.11,0.14,2.16,0.14,2.23
2,603711,605900,5,32,II,Domestic,NaN,NaN,NaN,NaN,0.110000,1.81,0.13000,2.01,0.130000,2.06,0.14,2.12,0.14,2.16,0.14,2.21,0.15,2.28
3,603710,605900,5,32,II,Domestic,NaN,NaN,NaN,NaN,0.030000,0.48,0.03000,0.53,0.030000,0.54,0.03,0.56,0.03,0.57,0.03,0.59,0.04,0.60
4,603700,605900,5,32,II,Domestic,NaN,NaN,NaN,NaN,0.008605,0.13,0.00957,0.14,0.009795,0.14,0.01,0.15,0.01,0.15,0.01,0.15,0.01,0.16


## Sum production/attraction tons for the 5 FAZs in SANDAG by mode

In [4]:
# Filter the data to include only records where either the origin or destination FAZ is located within San Diego.
faz_san_diego = faz_county[faz_county["County"] == "San Diego"]["FAZ"]
faf_sd_a = faf[faf["dms_dest"].isin(faz_san_diego)]
faf_sd_p = faf[faf["dms_orig"].isin(faz_san_diego)]
faf_sd_a.head()

,dms_orig,dms_dest,Mode,Commodity,Direction,Trade,fr_orig,fr_dest,fr_inmode,fr_outmode,distons_2017,disvalue_2017,distons_2025,disvalue_2025,distons_2030,disvalue_2030,distons_2035,disvalue_2035,distons_2040,disvalue_2040,distons_2045,disvalue_2045,distons_2050,disvalue_2050
246937,605901,607302,5,1,II,Domestic,NaN,NaN,NaN,NaN,0.000599,0.03,0.000771,0.04,0.000932,0.05,0.001094,0.06,0.001325,0.07,0.001604,0.09,0.001885,0.11
246938,603700,607302,5,1,II,Domestic,NaN,NaN,NaN,NaN,0.001015,0.06,0.001308,0.07,0.001580,0.09,0.001855,0.11,0.002246,0.13,0.002720,0.16,0.003196,0.19
246939,611100,607302,5,1,II,Domestic,NaN,NaN,NaN,NaN,0.001001,0.05,0.001289,0.07,0.001557,0.09,0.001828,0.10,0.002214,0.13,0.002681,0.16,0.003151,0.18
246940,607102,607302,5,1,II,Domestic,NaN,NaN,NaN,NaN,0.001091,0.06,0.001406,0.08,0.001699,0.10,0.001994,0.11,0.002415,0.14,0.002924,0.17,0.003436,0.20
246941,607101,607302,5,1,II,Domestic,NaN,NaN,NaN,NaN,0.000621,0.03,0.000800,0.04,0.000966,0.05,0.001135,0.06,0.001374,0.08,0.001664,0.09,0.001955,0.11


In [5]:
# Groupby and sum for both dataframes
result_a = faf_sd_a.groupby(['Mode'], as_index=False).agg(ton_a = ('distons_2017', 'sum'))
result_p = faf_sd_p.groupby(['Mode'], as_index=False).agg(ton_p = ('distons_2017', 'sum'))

# Reset index and drop the previous index column
result_a.reset_index(drop=True, inplace=True)
result_p.reset_index(drop=True, inplace=True)

# Concatenate results into a single DataFrame
ton_by_mode_sd = pd.merge(result_a, result_p, how="inner", on='Mode')
ton_by_mode_sd.to_csv('ton_by_mode_sd.csv', index=False)
ton_by_mode_sd.head()

,Mode,ton_a,ton_p
0,1,58080.092234,51036.043301
1,2,1134.082740,512.871564
2,3,338.982006,229.514966
3,4,39.474304,31.822049
4,5,1658.248343,640.953506


## Cleanup FAF data
1. Filter some truck-related modes
2. Delete unnecessary columns


In [6]:
# include some modes
mode_to_include = othermode_truck.set_index('Mode_Num')['Percentage'].to_dict()
# Filter out rows where 'Mode' is not in the mode_to_include
df = faf[faf['Mode'].isin(mode_to_include.keys())]
df['truck_perc'] = df['Mode'].map(mode_to_include) #assign percentage of truck by mode
df['ton'] = df['distons_2017'] * df['truck_perc']
# drop truck_perc column
df.drop('truck_perc', axis=1, inplace=True)

# Delete unnecessary columns
delete_col = ['distons_2017', 'disvalue_2017', 'Mode', 'fr_orig', 'fr_dest', 'fr_inmode', 'fr_outmode', 'Direction', 'Trade']
df.drop(delete_col, axis=1, inplace=True)

df.head()

C:\Users\mhasani\AppData\Local\Temp/ipykernel_17016/1540953186.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['truck_perc'] = df['Mode'].map(mode_to_include) #assign percentage of truck by mode
C:\Users\mhasani\AppData\Local\Temp/ipykernel_17016/1540953186.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ton'] = df['distons_2017'] * df['truck_perc']
C:\Users\mhasani\AppData\Local\Temp/ipykernel_17016/1540953186.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,dms_orig,dms_dest,Commodity,distons_2025,disvalue_2025,distons_2030,disvalue_2030,distons_2035,disvalue_2035,distons_2040,disvalue_2040,distons_2045,disvalue_2045,distons_2050,disvalue_2050,ton
0,605901,605900,32,0.01000,0.18,0.010000,0.18,0.01,0.19,0.01,0.19,0.01,0.20,0.01,0.20,0.002500
1,605900,605900,32,0.13000,1.97,0.130000,2.01,0.13,2.07,0.14,2.11,0.14,2.16,0.14,2.23,0.027500
2,603711,605900,32,0.13000,2.01,0.130000,2.06,0.14,2.12,0.14,2.16,0.14,2.21,0.15,2.28,0.027500
3,603710,605900,32,0.03000,0.53,0.030000,0.54,0.03,0.56,0.03,0.57,0.03,0.59,0.04,0.60,0.007500
4,603700,605900,32,0.00957,0.14,0.009795,0.14,0.01,0.15,0.01,0.15,0.01,0.15,0.01,0.16,0.002151


## Sum production/attraction tons for the 5 FAZs in SANDAG by 15 aggregated commodities

### Aggregate Commodity Level

In [7]:
# 1. Assign SANDAG commodity groups based on SCTG commodity group
# Create a dictionary to map Commodity values to CG values
commodity_to_cg = commodity_group.set_index('SCTG')['CG'].to_dict()

# Use the map function to directly assign CG values
df['CG'] = df['Commodity'].map(commodity_to_cg)

# drop commodity column
df.drop('Commodity', axis=1, inplace=True)

# 2. Aggregate the Tonnage Data by Origin/Dest and Commodity Group
df = df.groupby(['dms_orig', 'dms_dest', 'CG'], as_index=False).agg({'ton': 'sum'})

df.head()

C:\Users\mhasani\AppData\Local\Temp/ipykernel_17016/397591770.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CG'] = df['Commodity'].map(commodity_to_cg)
C:\Users\mhasani\AppData\Local\Temp/ipykernel_17016/397591770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('Commodity', axis=1, inplace=True)


,dms_orig,dms_dest,CG,ton
0,11,1204,CG-10,0.010000
1,11,1204,CG-11,0.025462
2,11,1204,CG-12,0.003779
3,11,1204,CG-13,0.004460
4,11,1204,CG-14,0.007220


In [8]:
# Filter the data to include only records where either the origin or destination FAZ is located within San Diego.
df_sd_a = df[df["dms_dest"].isin(faz_san_diego)]
df_sd_p = df[df["dms_orig"].isin(faz_san_diego)]
df_sd_a.head()

,dms_orig,dms_dest,CG,ton
617,11,607302,CG-11,0.011121
618,11,607302,CG-13,0.007824
619,11,607302,CG-14,0.003461
620,11,607302,CG-2,0.001582
621,11,607302,CG-4,0.010000


In [9]:
# Groupby and sum for both dataframes
result_a = df_sd_a.groupby(['CG'], as_index=False).agg(ton_a = ('ton', 'sum'))
result_p = df_sd_p.groupby(['CG'], as_index=False).agg(ton_p = ('ton', 'sum'))

# Reset index and drop the previous index column
result_a.reset_index(drop=True, inplace=True)
result_p.reset_index(drop=True, inplace=True)

# Concatenate results into a single DataFrame
ton_by_cg_sd = pd.merge(result_a, result_p, how="inner", on='CG')
ton_by_cg_sd.to_csv('ton_by_cg_sd.csv', index=False)
ton_by_cg_sd.head()

,CG,ton_a,ton_p
0,CG-1,2527.258969,3222.786519
1,CG-10,9821.125889,7185.742797
2,CG-11,3016.239932,1441.671879
3,CG-12,3831.388536,4913.682697
4,CG-13,1341.382175,771.864751


### Identify if one end of a OD is in Orange County
This will be used later in the OD distance calculation

In [10]:
# Create a mapping dictionary for FAZ to County
faz_san_diego = faz_county[faz_county["County"] == "San Diego"]["FAZ"]
# create a list of FAZ outside of San Diego
faz_non_sd = faz_county[faz_county["County"] != "San Diego"]["FAZ"]

df['county_orig'] = np.nan
df['county_dest'] = np.nan

# if it's within San Diego
df.loc[df['dms_orig'].isin(faz_san_diego), 'county_orig'] = df.loc[df['dms_orig'].isin(faz_san_diego), 'dms_orig'].astype(str)   
df.loc[df['dms_dest'].isin(faz_san_diego), 'county_dest'] = df.loc[df['dms_dest'].isin(faz_san_diego), 'dms_dest'].astype(str)   

# if it's outside San Diego but within CA, call it Rest of California
df.loc[df['dms_orig'].isin(faz_non_sd), 'county_orig'] = "Rest of California"
df.loc[df['dms_dest'].isin(faz_non_sd), 'county_dest'] = "Rest of California"

# Where county is NAN, it means the counts is outside of California
df.loc[df['county_orig'].isnull(), 'county_orig'] = 'Other States'
df.loc[df['county_dest'].isnull(), 'county_dest'] = 'Other States'

df.head()

,dms_orig,dms_dest,CG,ton,county_orig,county_dest
0,11,1204,CG-10,0.010000,Other States,Other States
1,11,1204,CG-11,0.025462,Other States,Other States
2,11,1204,CG-12,0.003779,Other States,Other States
3,11,1204,CG-13,0.004460,Other States,Other States
4,11,1204,CG-14,0.007220,Other States,Other States


In [11]:
# group by and summarize by origin and destination counties
truck_ton = df.groupby(['county_orig', 'county_dest'], as_index=False).agg(ton = ('ton', 'sum'))
truck_ton.to_csv('truck_ton.csv', index=False)
truck_ton.head()

,county_orig,county_dest,ton
0,607302,607302,781.466826
1,607302,607303,1392.614361
2,607302,607304,665.291648
3,607302,607305,956.361606
4,607302,607306,1400.163622


In [12]:
# long to wide format
truck_ton_matrix = truck_ton.pivot(index='county_orig', columns='county_dest', values='ton')
truck_ton_matrix.to_csv('truck_ton_matrix.csv')
truck_ton_matrix.head()

county_dest,607302,607303,607304,607305,607306,Other States,Rest of California
county_orig,,,,,,,
607302,781.466826,1392.614361,665.291648,956.361606,1400.163622,873.274623,1537.317967
607303,1392.614361,2493.814877,1187.110852,1709.568575,2524.290243,1655.360518,2770.551964
607304,665.291648,1187.110852,571.381795,819.615696,1196.990881,744.729663,1274.853117
607305,956.361606,1709.568575,819.615696,1178.639708,1730.638461,1100.743453,1851.256385
607306,1400.163622,2524.290243,1196.990881,1730.638461,2608.203937,1834.933229,2845.259428
